In [144]:
import pandas as pd

In [145]:
import numpy as  np

In [146]:
import emoji

In [147]:
import nltk

In [148]:
from nltk.corpus import stopwords

In [149]:
from nltk.stem.snowball import SnowballStemmer

In [220]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [151]:
import matplotlib.pyplot as plt 

In [152]:
from keras.utils import np_utils

In [153]:
sw = stopwords.words('english')
stem = SnowballStemmer('english')

In [155]:
emoji_dic = emoji.EMOJI_UNICODE

In [156]:
emoji.emojize(':1st_place_medal:')

'🥇'

In [157]:
url = 'https://raw.githubusercontent.com/coding-blocks-archives/ML28Dec2019PP/master/Day24-%20Word%20Embeddings/Emoji%20Prediction/train_emoji.csv'

In [158]:
df = pd.read_csv(url)

In [159]:
X_train = df.drop(columns=['Unnamed: 2','Unnamed: 3']).values

In [160]:
y_train = X_train[:,1]

In [161]:
X_train = X_train[:,0]

In [162]:
url_test = 'https://raw.githubusercontent.com/coding-blocks-archives/ML28Dec2019PP/master/Day24-%20Word%20Embeddings/Emoji%20Prediction/test_emoji.csv'

In [163]:
df_test = pd.read_csv(url_test)

In [164]:
X_test = df_test.values[:,0]

In [165]:
for i in range(X_test.shape[0]):
    X_test[i] = X_test[i][:-1]

In [166]:
y_test = df_test.values[:,1]

In [167]:
y_test

array([3, 2, 0, 2, 0, 0, 0, 4, 2, 1, 3, 3, 2, 1, 3, 3, 2, 3, 4, 0, 2, 4,
       3, 3, 3, 3, 0, 2, 0, 0, 1, 3, 2, 0, 1, 2, 4, 4, 2, 2, 2, 0, 1, 2,
       0, 2, 2, 2, 3, 3, 0, 3, 2, 2, 3], dtype=object)

In [168]:
def create_dataset(X):
    for i in range(X.shape[0]):
        X[i]= [word for word in X[i].split()]
    return X

In [169]:
X_test = create_dataset(X_test)

In [170]:
X_train = create_dataset(X_train)

In [171]:
vocab = set()
#print(X_train)
for sentence in X_train:
    for word in sentence:
        vocab.add(word)        

In [172]:
maximize = -1
for i in X_train:
    if len(i) > maximize:
        maximize = len(i)

In [175]:
file = open('glove.6B.50d.txt',encoding="utf8")

In [176]:
text = file.read()

In [177]:
text = text.split('\n')

In [181]:
dictionary = dict()
for i in range(len(text)):
    sent = text[i].split(' ')
    vector =  " ".join(sent[1:])
    embed_vec = []
    for i in vector.split():
        embed_vec.append(float(i))
    dictionary[sent[0]] = embed_vec 
    

In [182]:
def create_embedded_dataset():
    embedded_output = np.zeros((X_train.shape[0],10,50))
    
    for i in range(X_train.shape[0]):
        for j in range(len(X_train[i])):
            embedded_output[i][j] = dictionary[X_train[i][j].lower()]
    return embedded_output        

In [183]:
X_train = create_embedded_dataset()

In [185]:
emoji_dictionary = {
    "0" : "\u2764\uFE0F",
    "1" : ":baseball:",
    "2" : ":beaming_face_with_smiling_eyes:",
    "3" : ":downcast_face_with_sweat:",
    "4" : ":fork_and_knife:"
}

In [187]:
y_train = np_utils.to_categorical(y_train)

## Model creation

In [188]:
from keras.layers import *

In [189]:
from keras.models import Sequential

In [209]:
model = Sequential()

model.add(LSTM(64 , input_shape = (10,50) , return_sequences = True))
model.add(LSTM(64 , return_sequences = False))
model.add(Dense(5 , activation = 'softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________________________


In [210]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [211]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('best_model.h5' , monitor = 'val_accuracy' , verbose = True , save_best_only = True)
#earlystop = EarlyStopping(monitor = 'val_loss' , patience=10)
hist = model.fit(X_train , y_train , epochs= 21 , validation_split=0.1 , batch_size=32 , callbacks=[checkpoint])

Train on 117 samples, validate on 14 samples
Epoch 1/21
117/117 [==============================] - 1s 7ms/step - loss: 1.5781 - accuracy: 0.2650 - val_loss: 1.5766 - val_accuracy: 0.3571

Epoch 00001: val_accuracy improved from -inf to 0.35714, saving model to best_model.h5
Epoch 2/21
117/117 [==============================] - 0s 546us/step - loss: 1.5057 - accuracy: 0.3333 - val_loss: 1.5982 - val_accuracy: 0.3571

Epoch 00002: val_accuracy did not improve from 0.35714
Epoch 3/21
117/117 [==============================] - 0s 529us/step - loss: 1.4600 - accuracy: 0.3846 - val_loss: 1.5917 - val_accuracy: 0.4286

Epoch 00003: val_accuracy improved from 0.35714 to 0.42857, saving model to best_model.h5
Epoch 4/21
117/117 [==============================] - 0s 537us/step - loss: 1.4045 - accuracy: 0.4444 - val_loss: 1.5176 - val_accuracy: 0.2857

Epoch 00004: val_accuracy did not improve from 0.42857
Epoch 5/21
117/117 [==============================] - 0s 494us/step - loss: 1.3302 - accur

In [212]:
model.load_weights('best_model.h5')

In [213]:
def create_embedded_dataset_test():
    embedded_output = np.zeros((len(X_test),10,50))
    
    for i in range(len(X_test):
        for j in range(len(X_test[i])):
            embedded_output[i][j] = dictionary[X_test[i][j].lower()]
    return embedded_output

In [214]:
data_test = create_embedded_dataset_test()

In [215]:
predict = model.predict(data_test)

In [216]:
j =0 
for i in X_test:
    sent = " ".join(i)
    print(sent)
    print(emoji.emojize(emoji_dictionary[str(np.argmax(predict[j]))]))
    j+=1
    

he did not answer
😓
he got a raise
😓
she got me a present
😓
ha ha ha it was so funny
😁
he is a good friend
😁
I am upset
😓
We had such a lovely dinner tonight
😁
where is the food
🍴
Stop making this joke ha ha ha
😁
where is the ball
⚾
work is hard
😁
This girl is messing with me
❤️
are you serious ha ha
😓
Let us go play baseball
⚾
This stupid grader is not working
😓
work is horrible
😁
Congratulation for having a baby
😁
stop messing around
😓
any suggestions for dinner
😁
I love taking breaks
❤️
you brighten my day
❤️
I boiled rice
🍴
she is a bully
❤️
Why are you feeling bad
😓
I am upset
😓
I worked during my birthday
😁
My grandmother is the love of my life
❤️
enjoy your break
🍴
valentine day is near
😁
I miss you so much
❤️
throw the ball
⚾
My life is so boring
❤️
she said yes
😁
will you be my valentine
❤️
he can pitch really well
⚾
dance with me
😁
I am starving
🍴
See you at the restaurant
🍴
I like to laugh
😁
I will go dance
🍴
I like your jacket
❤️
i miss her
❤️
what is your favorite baseball